In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Forward and backward passes

In [2]:
#export
from exp.nb_01 import *
def get_data():
    path = datasets.download_data(MNIST_URL, ext = '.gz')
    with gzip.open(path,'rb') as f:
        ((x_train,y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor,(x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-7.6999e-06), tensor(1.))

In [7]:
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"near zero {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

## Foundations version
### Basic architecture
One hidden layer  
Ten activations for final output 

In [10]:
# number of hidden notes
nh = 50

In [11]:
# simplified kaiming init/ he init
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)

w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# this should be E~(0,1), (mean, std)
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [14]:
def lin(x, w, b): return x@w+b

In [15]:
t = lin(x_valid, w1, b1)
t.mean(), t.std()

(tensor(-0.0086), tensor(0.9536))

In [16]:
def relu(x): return x.clamp_min(0.) # replace the negatives data to 0

In [17]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.3690), tensor(0.5745))

In [18]:
# kaiming init/ he init for relu
w1 = torch.randn(m, nh)*math.sqrt(2./m)
w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0505))

In [19]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.4717), tensor(0.7712))

In [20]:
#export
from torch.nn import init

In [21]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode = 'fan_out') # fan_in: forward pass, fan_out: backward pass
t = relu(lin(x_valid, w1, b1))

In [22]:
# init.kaiming_normal_?? # F means torch.nn.founctional

In [23]:
w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0506))

In [24]:
t.mean(), t.std()

(tensor(0.5478), tensor(0.8065))

In [25]:
w1.shape # there it returns to output = input.matmul(weight.t()) weight transpose 

torch.Size([784, 50])

In [26]:
# a new activation
# what if ...?
def relu(x): return x.clamp_min(0.)-0.5

In [27]:
# kaiming init/ he init for relu
w1 = torch.randn(m, nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0817), tensor(0.8370))

In [28]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [29]:
%timeit -n 10 _=model(x_valid)

4.99 ms ± 575 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

## Loss function: MSE
To simplify things, mean sqaure error (MSE) is used here

In [31]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. `mse` is not a suitable loss funtion for multi-class classification; we will use a better loss function soon. 

In [32]:
def mse(output, targ): return (output.squeeze(-1)-targ).pow(2).mean() # 1 or -1 to keep it squeeze to array

In [33]:
y_train, y_valid = y_train.float(), y_valid.float()

In [34]:
preds = model(x_train) # predictions

In [35]:
mse(preds, y_train)

tensor(24.1224)

## Gradients and backward pass

In [36]:
def mse_grad(inp, targ):
    # gradient of loss with respect to output of previous output layer
    inp.g = 2. *  (inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0]

In [37]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g # for relu, <0: gradient = 0, >0: gradient = 1

In [38]:
def lin_grad(inp, out, w, b):
    # grad of matmul with prespect to input
    inp.g = out.g @ w.t() # t: transpose
    w.g = (inp.unsqueeze(-1)*out.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [39]:
def forward_and_backward(inp, targ):
    # forward pass
    l1 = inp @ w1 +b1
    l2 = relu(l1)
    out = l2 @ w2 +b2
    # loss is actually not needed in backward
    loss = mse(out, targ)
    
    # backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [40]:
forward_and_backward(x_train, y_train)

In [41]:
# save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone() # save the gradients for the optimizer

In [42]:
#######################################

In [43]:
# Cheat a little bit and use Pytorch autograd to check our results
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [44]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 +b22
    return mse(out, targ)

In [45]:
loss = forward(xt2, y_train)

In [46]:
loss.backward()

## Layers as classes

In [47]:
class Relu():
    def __call__ (self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [48]:
class Lin():
    def __init__(self, w, b):
        self. w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # creating a giant outer product, just to sum it, is not efficient
        self.w.g = (self.inp.unsqueeze(-1)*self.out.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [49]:
class MSE():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze(1)-targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze()-self.targ).unsqueeze(-1)/self.targ.shape[0]

In [50]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [51]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [52]:
%time loss = model(x_train, y_train)

CPU times: user 125 ms, sys: 31.2 ms, total: 156 ms
Wall time: 33.2 ms


In [53]:
%time model.backward() # it is slow

CPU times: user 5.48 s, sys: 15.7 s, total: 21.2 s
Wall time: 6.18 s


## Module.forward() 

In [54]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented, LOKTAR')
    def backward(self): self.bwd(self.out, *self.args)

In [55]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float()*out.g

In [56]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi, bj -> ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [57]:
class MSE(Module):
    def forward(self, inp, targ): return (inp.squeeze()-targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2. * (inp.squeeze()-targ).unsqueeze(-1)/targ.shape[0]

In [58]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [59]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [60]:
%time loss = model(x_train, y_train)

CPU times: user 188 ms, sys: 15.6 ms, total: 203 ms
Wall time: 36.6 ms


In [61]:
%time model.backward() # faster, einsten

CPU times: user 219 ms, sys: 781 ms, total: 1 s
Wall time: 352 ms


## Without the einsum 

In [62]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [63]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [64]:
%time loss = model(x_train, y_train)

CPU times: user 188 ms, sys: 31.2 ms, total: 219 ms
Wall time: 32.6 ms


In [65]:
%time model.backward() # faster, einsten same as matmul operator

CPU times: user 172 ms, sys: 703 ms, total: 875 ms
Wall time: 334 ms


## nn.Linear and nn.Module 

In [66]:
#export
from torch import nn

In [67]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(),targ)

In [68]:
model = Model(m, nh, 1)

In [69]:
%time loss = model(x_train, y_train)

CPU times: user 93.8 ms, sys: 46.9 ms, total: 141 ms
Wall time: 33.4 ms


In [70]:
%time loss.backward()

CPU times: user 156 ms, sys: 156 ms, total: 312 ms
Wall time: 43.7 ms
